In [1]:
import pandas as pd
import numpy as np
from sympy import Matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import skew, kurtosis, norm
import seaborn as sns
import matplotlib.pyplot as plt
from Functions import *

# Homework 7

# 1 GMO

## 1.1 GMO's Approach


##### (a) Why does GMO believe they can more easily predict long-run than short-run asset class performance?

In the short-run, valuations are more volatile and uncertain. Issues like the agency problem, wherein agent-principal relationship may translate into incentives to maintain/foster deviations from fundamental values, distort prices and are dificcult to predict. Conversely, in the long run, prices tend to meet fundamental values, which GMO believe they understand and can predict with much greater confidence.

##### (b) What predicting variables does the case mention are used by GMO? Does this fit with the goal of long-run forecasts?

The case mentions (1) dividend yield, (2) price/earnings multiple variation, (3) profit margin variaitons, and (4) sales per share variations. This fits their view of long-run forecasts in that these variables are likely to filter out short-run anomalies in the long run and approach their fundamental value. 

##### (c) How has this approach led to contrarian positions?

GMO believes that in the short-term prices may deviate from their fundamental value, and as they construct their fundamental forecasts from the variables mentioned, among others, their valuations may differ from the ones observed in the short term. Consequently, at times, GMO has produced forecasts that seem to contradict observed market behavior (which GMO will likely say is driven by short-term incentives/fluctuations).

##### (d) How does this approach raise business risk and managerial career risk?

When deviation from short-term trends in the market, and especially when those deviations are translating into losses, clients become more likely to cease business relationships, or reallocate funds away from underperforming managers. If withdrawals or client losses become large enough, it may translate into a substantial business risk. Similarly, the cost of deviating from investment trends may be high for an investment professional's career.



## 2. Analyzing GMO


In [2]:
dfs=pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='signals')
dfs.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
dfret=pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='returns (total)')
dfret.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
dfrf=pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='risk-free rate')
dfrf.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
dfretx=dfret.copy()
dfretx[['SPY', 'GMWAX']] = dfretx[['SPY', 'GMWAX']].sub(dfrf['US3M'], axis=0)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

### 2.1 Calculate the mean, volatility, and Sharpe ratio for GMWAX.

In [3]:
subsamples=[(1996,2011),(2012,2023),(1996,2023)]
res=pd.DataFrame()
for i in subsamples:
    tstats=calculate_statistics_array(dfretx[(dfretx['Date']>=str(i[0])) & (dfretx['Date']<=str(i[1]+1))]['GMWAX'])
    res.loc[str(i),'mean']=tstats['mean']
    res.loc[str(i),'vol']=tstats['volatility']
    res.loc[str(i),'Sharpe']=tstats['sharpe']
res

,mean,vol,Sharpe
"(1996, 2011)",0.0158,0.1247,0.1270
"(2012, 2023)",0.0364,0.0942,0.3869
"(1996, 2023)",0.0249,0.1124,0.2212


### 2.2 GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. For all three samples, analyze extreme scenarios by looking at

In [4]:
res=pd.DataFrame()
for i in subsamples:
    tstats=calculate_statistics(dfret[(dfret['Date']>=str(i[0])) & (dfret['Date']<=str(i[1]+1))])['GMWAX']
    tstatsx=calculate_statistics(dfretx[(dfretx['Date']>=str(i[0])) & (dfretx['Date']<=str(i[1]+1))])['GMWAX']
    res.loc[str(i),'Min Return']=dfret[(dfret['Date']>=str(i[0])) & (dfret['Date']<=str(i[1]+1))]['GMWAX'].min()
    res.loc[str(i),'VaR 5th']=tstats['VaR']
    res.loc[str(i),'Maximum Drawdown']=tstatsx['Max_Drawdown']['max_drawdown']
    res.loc[str(i),'Max Drawdown Start']=tstatsx['Max_Drawdown']['max_drawdown_start_date']
    res.loc[str(i),'Max Drawdown Min']=tstatsx['Max_Drawdown']['max_drawdown_min_date']
    res.loc[str(i),'Max Drawdown Recovery']=tstatsx['Max_Drawdown']['max_recovery_date']
res
    

,Min Return,VaR 5th,Maximum Drawdown,Max Drawdown Start,Max Drawdown Min,Max Drawdown Recovery
"(1996, 2011)",-0.1450,-0.0562,3.6572,1998-02-28,1998-08-31,1999-04-30
"(2012, 2023)",-0.1186,-0.0368,3.2220,2019-01-31,2020-03-31,2022-11-30
"(1996, 2023)",-0.1450,-0.0468,3.6572,1998-02-28,1998-08-31,1999-04-30


a) Not too bad

b) Not much

In [5]:
res=pd.DataFrame()

# iterate over subsamples and append results to dataframe
for i in subsamples:
    tdf=dfretx[(dfretx['Date']>=str(i[0])) & (dfretx['Date']<=str(i[1]+1))].copy()
    tdf.dropna(inplace=True)
    model=LinearRegression()
    model.fit(np.array(tdf['SPY']).reshape(-1,1), np.array(tdf['GMWAX']).reshape(-1,1))
    alpha = model.intercept_[0]*12
    beta = model.coef_[0][0]
    r2 = r2_score(tdf['GMWAX'], model.predict(np.array(tdf['SPY']).reshape(-1,1)))
    res.loc[str(i),'alpha']=alpha
    res.loc[str(i),'beta']=beta
    res.loc[str(i),'r2']=r2

res


,alpha,beta,r2
"(1996, 2011)",-0.0058,0.5396,0.5071
"(2012, 2023)",-0.0327,0.5738,0.7544
"(1996, 2023)",-0.0166,0.5506,0.5821


In [6]:
res=pd.DataFrame(columns=['alpha','beta_DP','beta_EP','beta_US10Y','r2'])

tdf=dfretx.copy()
tdf = pd.merge(tdf, dfs, on='Date')
tdf['SPY'] = tdf['SPY'].shift(-1)
tdf.drop(columns=['GMWAX'], inplace=True)
tdf.dropna(inplace=True)

case1='DP'
model_DP=LinearRegression()
model_DP.fit(np.array(tdf['DP']).reshape(-1,1), np.array(tdf['SPY']).reshape(-1,1))
res.loc[case1,'alpha']=model_DP.intercept_[0]*12
res.loc[case1,'beta_DP']=model_DP.coef_[0][0]
res.loc[case1,'r2']=r2_score(tdf['SPY'], model_DP.predict(np.array(tdf['DP']).reshape(-1,1)))
pred_DP=model_DP.predict(np.array(tdf['DP']).reshape(-1,1))

case2='EP'
model_EP=LinearRegression()
model_EP.fit(np.array(tdf['EP']).reshape(-1,1), np.array(tdf['SPY']).reshape(-1,1))
res.loc[case2,'alpha']=model_EP.intercept_[0]*12
res.loc[case2,'beta_EP']=model_EP.coef_[0][0]
res.loc[case2,'r2']=r2_score(tdf['SPY'], model_EP.predict(np.array(tdf['EP']).reshape(-1,1)))
pred_EP=model_EP.predict(np.array(tdf['EP']).reshape(-1,1))

case3='DP+EP+US10Y'
model_3=LinearRegression()
model_3.fit(np.array(tdf[['DP','EP','US10Y']]), np.array(tdf['SPY']).reshape(-1,1))
res.loc[case3,'alpha']=model_3.intercept_[0]*12
res.loc[case3,'beta_DP']=model_3.coef_[0][0]
res.loc[case3,'beta_EP']=model_3.coef_[0][1]
res.loc[case3,'beta_US10Y']=model_3.coef_[0][2]
res.loc[case3,'r2']=r2_score(tdf['SPY'], model_3.predict(np.array(tdf[['DP','EP','US10Y']])))
pred_3=model_3.predict(np.array(tdf[['DP','EP','US10Y']]))

tspy=tdf['SPY']


res.fillna('')

,alpha,beta_DP,beta_EP,beta_US10Y,r2
DP,-0.1530,0.0102,,,0.0107
EP,-0.0912,,0.0031,,0.0081
DP+EP+US10Y,-0.1780,0.0089,0.0026,-0.0018,0.0209


In [7]:
res=pd.DataFrame()
tdf=pd.DataFrame(
    {
        'Date':dfret['Date'].loc[tspy.index],
        'DP':(pred_DP.reshape(1, -1)*100*np.array(tspy).reshape(1,-1))[0],
        'EP':(pred_EP.reshape(1, -1)*100*np.array(tspy).reshape(1,-1))[0],
        'DP+EP+US10Y':(pred_3.reshape(1, -1)*100*np.array(tspy).reshape(1,-1))[0],
        'SPY':tspy
        }
)
tstats=calculate_market_statistics(tdf.drop(['SPY'],axis=1), 
                                   tdf['SPY'])
ttstats=calculate_statistics(tdf).transpose()

tstats.merge(ttstats, left_index=True, right_index=True)

,alpha,market_beta,treynor_ratio,information_ratio,r2,mean,volatility,sharpe,skewness,kurtosis,VaR,CVaR,Max_Drawdown
DP,0.0239,0.6617,0.1155,0.3000,0.6061,0.0764,0.1267,0.6029,-0.2719,12.5107,-0.0440,-0.0824,"{'max_drawdown': 4.669559123083582, 'max_drawd..."
EP,0.0277,0.5384,0.1309,0.4299,0.6080,0.0705,0.1029,0.6845,0.3153,3.4377,-0.0410,-0.0604,"{'max_drawdown': 2.456084644300279, 'max_drawd..."
DP+EP+US10Y,0.0514,0.6001,0.1651,0.4978,0.4289,0.0991,0.1366,0.7250,0.4002,5.8860,-0.0527,-0.0862,"{'max_drawdown': 3.489055880332835, 'max_drawd..."


In [8]:
res=pd.DataFrame()

ttdf=tdf[(tdf['Date']>=str(2000)) & (tdf['Date']<=str(2012))].copy()
ttdf=ttdf.merge(dfrf, on='Date', how='left')

tstats=calculate_market_statistics(ttdf.drop(['SPY'],axis=1), 
                                   ttdf['SPY'])
ttstats=calculate_statistics(ttdf).transpose()

tstats.merge(ttstats, left_index=True, right_index=True)

,alpha,market_beta,treynor_ratio,information_ratio,r2,mean,volatility,sharpe,skewness,kurtosis,VaR,CVaR,Max_Drawdown
DP,0.0303,0.7766,0.0425,0.2850,0.5844,0.0330,0.1650,0.2002,-0.1731,9.9049,-0.0520,-0.1216,"{'max_drawdown': 4.825122607545482, 'max_drawd..."
EP,0.0237,0.4352,0.0579,0.2747,0.4020,0.0252,0.1115,0.2261,0.9554,5.7216,-0.0396,-0.0639,"{'max_drawdown': 2.6002328805533153, 'max_draw..."
DP+EP+US10Y,0.0631,0.5658,0.1150,0.5163,0.3613,0.0651,0.1529,0.4256,0.3295,5.6494,-0.0651,-0.1039,"{'max_drawdown': 4.8641606953684455, 'max_draw..."
US3M,0.0231,-0.0036,-6.4460,4.0232,0.0102,0.0231,0.0058,4.0005,0.5140,-1.1074,0.0000,0.0000,"{'max_drawdown': 0.9984326018808778, 'max_draw..."


In [9]:
len([i for i in pred_3 if i>0])

310

Strat seems like more risk.

In [10]:
tdf=dfretx.copy()
tdf = pd.merge(tdf, dfs, on='Date')
tdf['SPY'] = tdf['SPY'].shift(-1)
tdf.dropna(inplace=True)
tdf['Error_OOS']=np.nan
tdf['Prediction_OOS']=np.nan
for i in range(61, len(tdf)):
    tmodel=LinearRegression()
    tmodel.fit(tdf[['DP','EP','US10Y']].iloc[i-60:i], np.array(tdf['SPY']).reshape(-1,1)[i-60:i])
    tpred=tmodel.predict(tdf[['DP','EP','US10Y']].iloc[i:i+1])
    tdf.iloc[i,6]=tdf['SPY'].iloc[i]-tpred[0][0]
    tdf.iloc[i,7]=tpred[0][0]

tdf.dropna(inplace=True)

print('OOS R2:  ',round(1-(sum([i**2 for i in tdf['Error_OOS']])/sum([(i-np.mean(tdf['SPY']))**2 for i in tdf['SPY']])),4))

OOS R2:   -0.212


In [11]:
tdf['Strategy_Return']=100*tdf['Prediction_OOS']*tdf['SPY']
tstats=calculate_market_statistics(tdf[['Date','Strategy_Return']], 
                                   tdf['SPY'])
ttstats=calculate_statistics(tdf).transpose()

tstats=tstats.merge(ttstats, left_index=True, right_index=True)
tstats[['Max_Drawdown', 'Max_Drawdown_Start', 'Max_Drawdown_Min', 'Max_Drawdown_Recovery']] = tstats['Max_Drawdown'].apply(pd.Series)
tstats

,alpha,information_ratio,market_beta,r2,treynor_ratio,mean,volatility,sharpe,skewness,kurtosis,VaR,CVaR,Max_Drawdown,Max_Drawdown_Start,Max_Drawdown_Min,Max_Drawdown_Recovery
Strategy_Return,0.1788,0.3395,-0.9101,0.0632,-0.1208,0.1099,0.5443,0.2019,1.0539,29.8956,-0.1285,-0.3342,15.5377,2002-03-31,2002-08-31,2009-01-31


In [12]:
print('Period:  ',str(2000),'-',str(2012))
ttdf=tdf[(tdf['Date']>=str(2000)) & (tdf['Date']<=str(2012))].copy()
tstats=calculate_market_statistics(ttdf[['Date','Strategy_Return']], 
                                   ttdf['SPY'])
ttstats=calculate_statistics(ttdf).transpose()

tstats=tstats.merge(ttstats, left_index=True, right_index=True)
tstats[['Max_Drawdown', 'Max_Drawdown_Start', 'Max_Drawdown_Min', 'Max_Drawdown_Recovery']] = tstats['Max_Drawdown'].apply(pd.Series)
tstats

Period:   2000 - 2012


,alpha,information_ratio,market_beta,r2,treynor_ratio,mean,volatility,sharpe,skewness,kurtosis,VaR,CVaR,Max_Drawdown,Max_Drawdown_Start,Max_Drawdown_Min,Max_Drawdown_Recovery
Strategy_Return,0.1980,0.2892,-1.7078,0.1342,-0.0886,0.1513,0.7356,0.2057,0.9311,17.8587,-0.1676,-0.4806,15.5377,2002-03-31,2002-08-31,2009-01-31


In [13]:
len([i for i in tdf['Prediction_OOS'] if i>0])

178